### Linear and Multilinear Functions

In [1]:
from gc_utils import *
# use vanilla algebra, so we're not bother by the sign of I*I.reverse()
alg = Algebra(4) 
locals().update(alg.blades)

In [2]:
#1.1 linear transformation
A = random_multivector(alg)
B = random_multivector(alg)
C = random_multivector(alg)
D = random_multivector(alg)
x = random_multivector(alg).grade(1)
a = random_multivector(alg).grade(1)
f = lambda x: (A*x*B+C*x*D).grade(1)
differential(f, x, a), f(a)

(0.802 𝐞₁ + 1.34 𝐞₂ + 1.66 𝐞₃ + -1.17 𝐞₄,
 0.802 𝐞₁ + 1.34 𝐞₂ + 1.66 𝐞₃ + -1.17 𝐞₄)

In [4]:
#1.2
g = lambda x: (C*x*D).grade(1)
g(f(a)), differential(g, f(a), f(a)), differential(g, 0, f(a))

(4.26 𝐞₁ + 2.97 𝐞₂ + -1.36 𝐞₃ + -1.12 𝐞₄,
 4.26 𝐞₁ + 2.97 𝐞₂ + -1.36 𝐞₃ + -1.12 𝐞₄,
 4.26 𝐞₁ + 2.97 𝐞₂ + -1.36 𝐞₃ + -1.12 𝐞₄)

In [5]:
# modify the input of the original skew_symmetrizer: a multivector Ar instead of vectors
def skew_symmetrizer(F, Ar, alg, h=1e-6, frame=None, r_frame=None):
    if not frame:
        frame = alg.frame
        r_frame = reciprocal(alg.frame)
    drF = 0
    r = Ar.grades[0]
    for base_vectors, reci_vectors in zip(permutations(frame, r), permutations(r_frame, r)):
        # if F is linear, vectors cause no difference in vectors_partial. The input is just Ar.
        # So actually we allow Ar being any multivector
        drF += (Ar.sp(wedge(base_vectors[::-1]))) * vectors_partial(F, np.zeros(r), reci_vectors, h=h)
    return (1/factorial(r)) * drF

In [6]:
#1.4 Extend a vector function to a multivector function.
# For linear f we ignore the position where f is measured, 
# just the differential at each direction.
def outermorphism(f, A: MultiVector, alg, h=1e-6, frame=None, r_frame=None):
    wf = lambda vectors: wedge([f(v) for v in vectors])
    outer = 0
    for r in A.grades:
        if r == 0:
            outer += A.grade(0)
            continue
        outer += skew_symmetrizer(wf, A.grade(r), alg, h, frame, r_frame)
    return outer

outermorphism(f, x, alg), f(x)

(-2.4 𝐞₁ + 5.69 𝐞₂ + 2.48 𝐞₃ + -4.61 𝐞₄,
 -2.4 𝐞₁ + 5.69 𝐞₂ + 2.48 𝐞₃ + -4.61 𝐞₄)

In [8]:
#1.6 A frame can be arbitrary as long as it represent the same space. 
# It's pretty handy to specify a domain.
h = 1e-2
A2 = e12
A2frame = [e1, e2]
A2r_frame = reciprocal(A2frame)
(
    outermorphism(f, P(A, A2), alg, h), 
    outermorphism(f, P(A, A2), alg, h, frame=A2frame, r_frame=A2r_frame), 
    outermorphism(f, A, alg, h, frame=A2frame, r_frame=A2r_frame),
)

(0.203 + -2.85 𝐞₁ + 1.29 𝐞₂ + 3.17 𝐞₃ + -3.22 𝐞₄ + 3.43 𝐞₁₂ + -7.09 𝐞₁₃ + 1.95 𝐞₁₄ + 7.02 𝐞₂₃ + -4.75 𝐞₂₄ + 5.84 𝐞₃₄,
 0.203 + -2.85 𝐞₁ + 1.29 𝐞₂ + 3.17 𝐞₃ + -3.22 𝐞₄ + 3.43 𝐞₁₂ + -7.09 𝐞₁₃ + 1.95 𝐞₁₄ + 7.02 𝐞₂₃ + -4.75 𝐞₂₄ + 5.84 𝐞₃₄,
 0.203 + -2.85 𝐞₁ + 1.29 𝐞₂ + 3.17 𝐞₃ + -3.22 𝐞₄ + 3.43 𝐞₁₂ + -7.09 𝐞₁₃ + 1.95 𝐞₁₄ + 7.02 𝐞₂₃ + -4.75 𝐞₂₄ + 5.84 𝐞₃₄)

In [7]:
#1.7 An outermorphism preserves outer products
vectors = random_r_vectors(3, alg)
outermorphism(f, wedge(vectors), alg), wedge([f(v) for v in vectors])

(-1.41 𝐞₁₂₃ + 0.481 𝐞₁₂₄ + 0.664 𝐞₁₃₄ + -2.19 𝐞₂₃₄,
 -1.41 𝐞₁₂₃ + 0.481 𝐞₁₂₄ + 0.664 𝐞₁₃₄ + -2.19 𝐞₂₃₄)

In [8]:
# By 1.7, splitting A into simple blades, we get a simpler outermorphism
# Here, the value at v replaces the slope at direction v by linearity
def outermorphism_(f, A: MultiVector, alg):
    outer = A.grade(0)
    for vectors in multi_frame_vectors(alg.frame):
        Vr = wedge(reciprocal(vectors)[::-1])
        outer += alg.sp(Vr,A) * wedge([f(v) for v in vectors]) # frame split of A
    return outer

outermorphism_(f, D, alg), outermorphism(f, D, alg)

(0.0419 + -0.29 𝐞₁ + 2.66 𝐞₂ + 6.35 𝐞₃ + -1.08 𝐞₄ + 16.6 𝐞₁₂ + -9.8 𝐞₁₃ + 13.1 𝐞₁₄ + 22.2 𝐞₂₃ + -30.1 𝐞₂₄ + 6.16 𝐞₃₄ + 40.4 𝐞₁₂₃ + -19.0 𝐞₁₂₄ + -31.0 𝐞₁₃₄ + 61.5 𝐞₂₃₄ + 73.9 𝐞₁₂₃₄,
 0.0419 + -0.29 𝐞₁ + 2.66 𝐞₂ + 6.35 𝐞₃ + -1.08 𝐞₄ + 16.6 𝐞₁₂ + -9.8 𝐞₁₃ + 13.1 𝐞₁₄ + 22.2 𝐞₂₃ + -30.1 𝐞₂₄ + 6.16 𝐞₃₄ + 40.4 𝐞₁₂₃ + -19.0 𝐞₁₂₄ + -31.0 𝐞₁₃₄ + 61.5 𝐞₂₃₄ + 73.9 𝐞₁₂₃₄)

In [9]:
#1.9 anoter outer product property
outermorphism(f, A.reverse(), alg), outermorphism(f, A, alg).reverse()

(0.375 + 0.995 𝐞₁ + 3.55 𝐞₂ + 2.28 𝐞₃ + 2.09 𝐞₄ + -5.74 𝐞₁₂ + 10.2 𝐞₁₃ + -7.17 𝐞₁₄ + 0.911 𝐞₂₃ + 6.69 𝐞₂₄ + -17.6 𝐞₃₄ + -52.1 𝐞₁₂₃ + 20.8 𝐞₁₂₄ + 38.1 𝐞₁₃₄ + -74.8 𝐞₂₃₄ + 38.9 𝐞₁₂₃₄,
 0.375 + 0.995 𝐞₁ + 3.55 𝐞₂ + 2.28 𝐞₃ + 2.09 𝐞₄ + -5.74 𝐞₁₂ + 10.2 𝐞₁₃ + -7.17 𝐞₁₄ + 0.911 𝐞₂₃ + 6.69 𝐞₂₄ + -17.6 𝐞₃₄ + -52.1 𝐞₁₂₃ + 20.8 𝐞₁₂₄ + 38.1 𝐞₁₃₄ + -74.8 𝐞₂₃₄ + 38.9 𝐞₁₂₃₄)

In [10]:
#1.10
fA2 = lambda x: P(f(x), A2)
outermorphism(fA2, A, alg), P(outermorphism(fA2, A, alg), A2)

(0.375 + 0.995 𝐞₁ + 3.55 𝐞₂ + 5.74 𝐞₁₂, 0.375 + 0.995 𝐞₁ + 3.55 𝐞₂ + 5.74 𝐞₁₂)

In [11]:
#1.11
def adjoint_outermorphism(f, A, alg, h=1e-6, frame=None, r_frame=None):
    F = lambda vectors: wedge([f(v) for v in vectors]).sp(A)
    outer = 0
    for r in A.grades:
        if r == 0:
            outer += A.grade(0)
            continue
        # Why np.zeros(r)? The derivative of a linear function is constant
        outer += simplicial_derivative(F, np.zeros(r), alg, h, frame, r_frame)
    return outer

In [12]:
#1.12
adjoint_outermorphism(fA2, P(A, A2), alg), adjoint_outermorphism(fA2, A, alg)

(0.375 + 0.942 𝐞₁ + 1.64 𝐞₂ + 1.62 𝐞₃ + -0.441 𝐞₄ + -5.61 𝐞₁₂ + -1.5 𝐞₁₃ + 2.7 𝐞₁₄ + 7.04 𝐞₂₃ + 2.08 𝐞₂₄ + 3.94 𝐞₃₄,
 0.375 + 0.942 𝐞₁ + 1.64 𝐞₂ + 1.62 𝐞₃ + -0.441 𝐞₄ + -5.61 𝐞₁₂ + -1.5 𝐞₁₃ + 2.7 𝐞₁₄ + 7.04 𝐞₂₃ + 2.08 𝐞₂₄ + 3.94 𝐞₃₄)

In [13]:
#1.13
# Exercise: implement the adjoint outermorphism by spliting A into vectors.
adjoint_outermorphism(f, A^B, alg), adjoint_outermorphism(f, A, alg) ^ adjoint_outermorphism(f, B, alg)

(0.13 + 2.13 𝐞₁ + 2.89 𝐞₂ + -0.866 𝐞₃ + 1.33 𝐞₄ + 3.76 𝐞₁₂ + -12.4 𝐞₁₃ + 4.93 𝐞₁₄ + 2.08 𝐞₂₃ + -2.62 𝐞₂₄ + 12.0 𝐞₃₄ + 1.68e+02 𝐞₁₂₃ + -72.1 𝐞₁₂₄ + 50.8 𝐞₁₃₄ + 1.56e+02 𝐞₂₃₄ + 16.9 𝐞₁₂₃₄,
 0.13 + 2.13 𝐞₁ + 2.89 𝐞₂ + -0.866 𝐞₃ + 1.33 𝐞₄ + 3.76 𝐞₁₂ + -12.4 𝐞₁₃ + 4.93 𝐞₁₄ + 2.08 𝐞₂₃ + -2.62 𝐞₂₄ + 12.0 𝐞₃₄ + 1.68e+02 𝐞₁₂₃ + -72.1 𝐞₁₂₄ + 50.8 𝐞₁₃₄ + 1.56e+02 𝐞₂₃₄ + 16.9 𝐞₁₂₃₄)

In [14]:
#1.14a r <= s
r, s = 2, 3
Ar = A.grade(r)
Bs = B.grade(s)
(
    Ar | adjoint_outermorphism(f, Bs, alg),
    outermorphism(f, Ar, alg) | Bs,
    adjoint_outermorphism(f, outermorphism(f, Ar, alg) | Bs, alg),
)

(-6.02 𝐞₁ + -3.93 𝐞₂ + -8.36 𝐞₃ + 6.98 𝐞₄,
 -14.7 𝐞₁ + -5.69 𝐞₂ + 3.34 𝐞₃ + 8.17 𝐞₄,
 -6.02 𝐞₁ + -3.93 𝐞₂ + -8.36 𝐞₃ + 6.98 𝐞₄)

In [15]:
#1.14b r >= s
r, s = 3, 2
Ar = A.grade(r)
Bs = B.grade(s)
(
    outermorphism(f, Ar | adjoint_outermorphism(f, Bs, alg), alg),
    Ar | adjoint_outermorphism(f, Bs, alg),
    outermorphism(f, Ar, alg) | Bs,
)

(8.33 𝐞₁ + -5.46 𝐞₂ + -73.3 𝐞₃ + 37.2 𝐞₄,
 -7.0 𝐞₁ + -0.914 𝐞₂ + 6.7 𝐞₃ + -3.15 𝐞₄,
 8.33 𝐞₁ + -5.46 𝐞₂ + -73.3 𝐞₃ + 37.2 𝐞₄)

In [16]:
#1.15 r = s
for r,s in [(1,1), (2,2), (3,3)]:
    Ar = A.grade(r)
    Bs = B.grade(s)
    print(
        Ar | adjoint_outermorphism(f, Bs, alg),
        outermorphism(f, Ar, alg) | Bs,
    )

3.35 3.35
-9.57 -9.57
7.35 7.35


In [17]:
# proof of 1.14a
r, s = 2, 3
Ar = A.grade(r)
Bs = B.grade(s)
frame = alg.frame
r_frame = reciprocal(alg.frame)
rhs = 0
r = Ar.grades[0]
F = lambda vectors: wedge([f(v) for v in vectors]) 
for base_vectors, reci_vectors in zip(permutations(frame, s), permutations(r_frame, s)):
    rhs += (Ar | (wedge(base_vectors[::-1]))) * (vectors_partial(F, np.zeros(s), reci_vectors, h=h) | Bs)
(
    Ar | adjoint_outermorphism(f, Bs, alg),
    rhs / factorial(s)
)

(-6.02 𝐞₁ + -3.93 𝐞₂ + -8.36 𝐞₃ + 6.98 𝐞₄,
 -6.02 𝐞₁ + -3.93 𝐞₂ + -8.36 𝐞₃ + 6.98 𝐞₄)

In [18]:
# singular, nonsingular functions
I = alg.pseudoscalar((1,))
# fA2 is singular
outermorphism(fA2, I, alg), outermorphism(f, I, alg), adjoint_outermorphism(f, I, alg)

(, 86.4 𝐞₁₂₃₄, 86.4 𝐞₁₂₃₄)

In [19]:
#1.16 Use inverse instead of reverse for negative signatures. case I = Ip
n = 4
Ir = I.inv()
F = lambda vectors: wedge([f(v) for v in vectors])
(
    simplicial_derivative(F, alg.frame, alg, h=1e-2), 
    Ir * outermorphism(f, I, alg), 
    adjoint_outermorphism(f, I, alg) * Ir
)

(86.4, 86.4, 86.4)

In [20]:
#1.16 case I != Ip
n = 2
I = e12
Ip = e23
Ir = I.inv()
Ipr = Ip.inv()
h = 1e-2
# a function from I to Ip
g = lambda x: P((A*P(x, I)*B).grade(1), Ip)
F = lambda vectors: wedge([g(v) for v in vectors])
(
    simplicial_derivative(F, alg.frame[:2], alg, h),
    Ir * outermorphism(g, I, alg, h),
    adjoint_outermorphism(g, Ip, alg, h) * Ipr
)

(-2.9 𝐞₁₃, -2.9 𝐞₁₃, -2.9 𝐞₁₃)

In [21]:
#1.17a
outermorphism(g, I, alg, h), Ip

(2.9 𝐞₂₃, 1 𝐞₂₃)

In [22]:
#1.17b Need extra minus sign for negative signature. You can try Algebra(3,1).
adjoint_outermorphism(g, Ip, alg, h), I

(2.9 𝐞₁₂, 1 𝐞₁₂)

In [23]:
# The relative orientation between I and I' can be arbitrary.
# Here we let the parent algebra determine the orientation.
def det(f, I, Ip):
    return (outermorphism(f, I, alg)/Ip)[0]

det(g, I, Ip)

2.901237247366156

In [24]:
#1.21a null part goes null
C = random_multivector(alg)
C_ = outermorphism(g, C, alg)
(1/det(g, I, Ip)) * adjoint_outermorphism(g, C_*Ip, alg) * Ir, C

(0.125 + 0.448 𝐞₁ + 0.646 𝐞₂ + 0.0231 𝐞₁₂,
 0.125 + 0.448 𝐞₁ + 0.646 𝐞₂ + 0.579 𝐞₃ + 0.248 𝐞₄ + 0.0231 𝐞₁₂ + 0.628 𝐞₁₃ + 0.158 𝐞₁₄ + 0.0517 𝐞₂₃ + 0.0384 𝐞₂₄ + 0.594 𝐞₃₄ + 0.27 𝐞₁₂₃ + 0.161 𝐞₁₂₄ + 0.439 𝐞₁₃₄ + 0.503 𝐞₂₃₄ + 0.496 𝐞₁₂₃₄)

In [25]:
#1.21b null part goes null
C_ = random_multivector(alg)
C = adjoint_outermorphism(g, C_, alg)
(1/det(g, I, Ip)) * outermorphism(g, C*I, alg) * Ipr, C_

(0.278 + 0.0417 𝐞₂ + 0.021 𝐞₃ + 0.222 𝐞₂₃,
 0.278 + 0.263 𝐞₁ + 0.0417 𝐞₂ + 0.021 𝐞₃ + 0.289 𝐞₄ + 0.137 𝐞₁₂ + 0.0585 𝐞₁₃ + 0.592 𝐞₁₄ + 0.222 𝐞₂₃ + 0.457 𝐞₂₄ + 0.922 𝐞₃₄ + 0.381 𝐞₁₂₃ + 0.0365 𝐞₁₂₄ + 0.326 𝐞₁₃₄ + 0.976 𝐞₂₃₄ + 0.389 𝐞₁₂₃₄)

In [26]:
#2.1 Note that characteristic multivectors are of even grades
r = 3  # try several r
h = 1e-2
vectors = random_r_vectors(r, alg)
F = lambda vectors: wedge([f(v) for v in vectors])
(
    F(vectors),
    outermorphism(f, wedge(vectors), alg, h),
    simplicial_derivative(F, vectors, alg, h), # characteristic multivectors
    derivative(lambda A: outermorphism(f, A, alg, h), wedge(vectors), alg, h, r),
)


(3.27 𝐞₁₂₃ + -1.65 𝐞₁₂₄ + 2.17 𝐞₁₃₄ + 3.85 𝐞₂₃₄,
 3.27 𝐞₁₂₃ + -1.65 𝐞₁₂₄ + 2.17 𝐞₁₃₄ + 3.85 𝐞₂₃₄,
 17.7 + 8.11 𝐞₁₂ + 6.26 𝐞₁₃ + 4.9 𝐞₁₄ + -1.48 𝐞₂₃ + 5.16 𝐞₂₄ + 3.77 𝐞₃₄,
 17.7 + 8.11 𝐞₁₂ + 6.26 𝐞₁₃ + 4.9 𝐞₁₄ + -1.48 𝐞₂₃ + 5.16 𝐞₂₄ + 3.77 𝐞₃₄)

In [27]:
#2.2
frame = random_r_vectors(len(alg.signature), alg)
r_frame = reciprocal(frame)
images = [f(v) for v in frame]
(
    sum(vr * b for b, vr in zip(images, r_frame)),
    derivative(f, x, alg)
)

(-5.13 + 0.75 𝐞₁₂ + 0.965 𝐞₁₃ + 0.398 𝐞₁₄ + -0.193 𝐞₂₃ + -0.0751 𝐞₂₄ + -1.16 𝐞₃₄,
 -5.13 + -3.12 𝐞₁ + 4.71 𝐞₂ + 1.96 𝐞₃ + 1.6 𝐞₄ + 1.27 𝐞₁₂ + 2.08 𝐞₁₃ + 3.08 𝐞₁₄ + 4.14 𝐞₂₃ + -0.692 𝐞₂₄ + 2.17 𝐞₃₄ + 1.68 𝐞₁₂₃ + 0.259 𝐞₁₂₄ + 2.67 𝐞₁₃₄ + 0.977 𝐞₂₃₄ + 3.56 𝐞₁₂₃₄)

In [28]:
#2.2 comes from linearity
images, [differential(f, a, v) for v in frame]

([0.0577 𝐞₁ + 1.5 𝐞₂ + 0.853 𝐞₃ + -0.887 𝐞₄,
  0.0398 𝐞₁ + 1.03 𝐞₂ + 0.979 𝐞₃ + -0.962 𝐞₄,
  0.91 𝐞₁ + 0.512 𝐞₂ + 0.194 𝐞₃ + 0.786 𝐞₄,
  0.665 𝐞₁ + 1.17 𝐞₂ + -0.619 𝐞₃ + 0.346 𝐞₄],
 [0.0577 𝐞₁ + 1.5 𝐞₂ + 0.853 𝐞₃ + -0.887 𝐞₄,
  0.0398 𝐞₁ + 1.03 𝐞₂ + 0.979 𝐞₃ + -0.962 𝐞₄,
  0.91 𝐞₁ + 0.512 𝐞₂ + 0.194 𝐞₃ + 0.786 𝐞₄,
  0.665 𝐞₁ + 1.17 𝐞₂ + -0.619 𝐞₃ + 0.346 𝐞₄])

In [29]:
#2.3 trace
def trace(f, alg):
    return sum(vr.sp(f(v)) for v, vr in zip(alg.frame, reciprocal(alg.frame)))
def divergence(f, alg):
    return sum(vr | f(v) for v, vr in zip(alg.frame, reciprocal(alg.frame)))
(
    derivative(f, e1, alg).grade(0),
    trace(f, alg),
    divergence(f, alg)
)

(-5.13, -5.13, -5.13)

In [30]:
#2.4
r = 2
h = 1e-2
F = lambda vectors: wedge([f(v) for v in vectors])
def char_multi_linear(f, r, alg: Algebra, frame=None, r_frame=None):
    if not frame:
        frame = alg.frame
        r_frame = reciprocal(frame)
    r_vec_frame = r_vector_frame_vectors(frame, r)
    r_reci_frame = r_vector_frame_vectors(r_frame, r, reverse=True)
    sdf = 0
    for vectors, r_vectors in zip(r_vec_frame, r_reci_frame):
        sdf += wedge(r_vectors[::-1]) * wedge([f(v) for v in vectors])
    return sdf

(
    char_multi_linear(f, r, alg),
    simplicial_derivative(F, np.zeros(r), alg, h)
)

(-1.56 + -5.41 𝐞₁₂ + -4.3 𝐞₁₃ + -2.97 𝐞₁₄ + -1.65 𝐞₂₃ + 0.358 𝐞₂₄ + -1.4 𝐞₃₄ + -0.874 𝐞₁₂₃₄,
 -1.56 + -5.41 𝐞₁₂ + -4.3 𝐞₁₃ + -2.97 𝐞₁₄ + -1.65 𝐞₂₃ + 0.358 𝐞₂₄ + -1.4 𝐞₃₄ + -0.874 𝐞₁₂₃₄)

In [31]:
#2.6 when full rank, only scalar remains
def trans2matrix(f, alg):
    values = [f(a) for a in alg.frame]
    return np.array([[ar.sp(v)[0] for v in values] for ar in reciprocal(alg.frame)])
matrix = trans2matrix(f, alg)
I = alg.pseudoscalar((1,))
char_multi_linear(f, 4, alg)[0], np.linalg.det(matrix), det(f, I, I)

(10.316418004503989, 10.316418004503996, 10.316418004503989)

In [32]:
#2.7 sum
r = 3
h = lambda x: f(x)+g(x)

# Sometimes I use a random frame to check frame independence
frame = random_r_vectors(len(alg.signature), alg)
r_frame = reciprocal(frame)
drF = 0
coef = [1/(factorial(r-s)*factorial(s)) for s in range(r+1)]
for base_vectors, reci_vectors in zip(permutations(frame, r), permutations(r_frame, r)):
    gh = 0
    for s in range(r+1):
        gh += coef[s]*wedge([f(v) for v in reci_vectors[:s]]) ^ wedge([g(v) for v in reci_vectors[s:]])
    drF += wedge(base_vectors[::-1]) * gh
char_multi_linear(h, r, alg), drF

(45.6 + 30.8 𝐞₁₂ + 24.6 𝐞₁₃ + 21.5 𝐞₁₄ + -0.567 𝐞₂₃ + 27.6 𝐞₂₄ + 20.9 𝐞₃₄,
 45.6 + 30.8 𝐞₁₂ + 24.6 𝐞₁₃ + 21.5 𝐞₁₄ + -0.567 𝐞₂₃ + 27.6 𝐞₂₄ + 20.9 𝐞₃₄)

In [33]:
#2.8 characteristic polynomial
def char_scalar(f, r, alg: Algebra, frame=None, r_frame=None):
    if not frame:
        frame = alg.frame
        r_frame = reciprocal(frame)
    r_vec_frame = r_vector_frame_vectors(frame, r)
    r_reci_frame = r_vector_frame_vectors(r_frame, r, reverse=True)
    sdf = 0
    for vectors, r_vectors in zip(r_vec_frame, r_reci_frame):
        sdf += alg.sp(wedge(r_vectors[::-1]), wedge([f(v) for v in vectors]))[0]
    return sdf

# This calculates n characteristic multivectors. 
# But drops the nonscalar information
def char_poly_coefs(f, alg, frame=None, r_frame=None):
    n = alg.d
    if frame:
        n = len(frame)
    return [(-1)**(n-s) * char_scalar(f, s, alg, frame, r_frame) for s in range(n+1)]

def polynomial(l, coefs):
    return sum(l ** i * coef for i, coef in enumerate(coefs[::-1]))

l = 3.3
n = 4
(
    char_multi_linear(lambda x: f(x) - l*x, n, alg), 
    polynomial(l, char_poly_coefs(f, alg))
)

(2.38e+02, 237.9149929311739)

In [34]:
def r_fold(f,r,value):
    return reduce(lambda v, _: f(v), range(r), value)

r_fold(f, 3, x), f(f(f(x))), r_fold(f, 0, x), x

(-4.83 𝐞₁ + 14.5 𝐞₂ + 19.0 𝐞₃ + -29.3 𝐞₄,
 -4.83 𝐞₁ + 14.5 𝐞₂ + 19.0 𝐞₃ + -29.3 𝐞₄,
 0.982 𝐞₁ + 0.636 𝐞₂ + 0.639 𝐞₃ + 0.552 𝐞₄,
 0.982 𝐞₁ + 0.636 𝐞₂ + 0.639 𝐞₃ + 0.552 𝐞₄)

In [35]:
#2.9 Cayley Hamilton
sum(r_fold(f, i, x) * coef for i, coef in enumerate(char_poly_coefs(f, alg)[::-1]))

-2.84e-14 𝐞₁ + -1.42e-14 𝐞₃ + -2.84e-14 𝐞₄

In [36]:
#2.10 We can find eigenvalues like the usual many-fold method.
# How about eigenvectors? 
# What else can we get from characteristic multivector? Some papers I should go through...
matrix = trans2matrix(f, alg)
eigvalues, eigvecs = np.linalg.eig(matrix)
coefs = char_poly_coefs(f, alg)
eigvalues, np.roots(coefs)

(array([ 1.37557497,  0.63099487, -4.48429983, -2.65047795]),
 array([-4.48429983, -2.65047795,  1.37557497,  0.63099487]))

In [37]:
# From Lasenby's paper, we can reconstruct a rotor from characteristic multivector:
# Note that the rotor must have non-zero scalar part
# the paper suggest such method is pretty robust for noisy data
R = blade_exp(random_r_blade(2, alg)) * blade_exp(random_r_blade(2, alg))
rotor = lambda x: R.sw(x) # a screw motion

terms_ratio(R.reverse(), sum(char_multi_linear(rotor, s, alg) for s in range(n+1)))

array([0.07003987, 0.07003987, 0.07003987, 0.07003987, 0.07003987,
       0.07003987, 0.07003987, 0.07003987])

In [38]:
# recovering matrix to transfromation
def matrix2trans(M, alg:Algebra ):
    d = alg.d
    assert M.shape[0] == d, 'dimension not fit'
    return lambda x: sum(c*e for c,e in zip(np.dot(M, x.asfullmv()[1:d+1]), alg.frame))
    
mf = matrix2trans(matrix, alg)
mf(x), f(x)

(0.868 𝐞₁ + 1.87 𝐞₂ + 1.39 𝐞₃ + -1.14 𝐞₄,
 0.868 𝐞₁ + 1.87 𝐞₂ + 1.39 𝐞₃ + -1.14 𝐞₄)

In [39]:
#2.11
s = 2
rhs = (1/s) * sum((-1)**(r+1) * char_scalar(f, s-r, alg) * trace(lambda x: r_fold(f, r, x), alg)[0] for r in range(1,s+1))
char_scalar(f, s, alg), rhs

(-1.5629113383782451, -1.5629113383782425)

In [40]:
#2.13
k = 3
char_scalar(f, k, alg), sum(np.prod(comb) for comb in combinations(eigvalues, k))

(17.65631192863215, 17.656311928632125)

In [41]:
#2.14 It's easier to show the eigenvalues of f^k are {lambda^k_i} by linearity
trace(lambda x: r_fold(f, k, x), alg)[0], sum(l**k for l in eigvalues)

(-105.94012930773613, -105.94012930773607)

In [42]:
# 3.4 A space must bound f to own an invariant subspace.
I = alg.pseudoscalar((1,))
outermorphism(f, I, alg), det(f, I, I)*I, adjoint_outermorphism(f, I, alg)

(10.3 𝐞₁₂₃₄, 10.3 𝐞₁₂₃₄, 10.3 𝐞₁₂₃₄)

In [43]:
#3.6
l = 2.2
n = 4
F = lambda x: f(x) - l*x
coefs = char_poly_coefs(f, alg)
outermorphism(F, I, alg), polynomial(l, coefs) * I

(41.9 𝐞₁₂₃₄, 41.9 𝐞₁₂₃₄)

In [44]:
#3.7
I = e12
frame = [e1,e2]
g = lambda x: P((A*x.grade(1)*B).grade(1), I)
coefs = char_poly_coefs(g, alg, frame, frame)
F = lambda x: g(x) - l*x
outermorphism(F, I, alg), polynomial(l, coefs) * I

(4.99 𝐞₁₂, 4.99 𝐞₁₂)

In [45]:
#3.8
I = e123
frame2 = [e3]
frame3 = [e1,e2,e3]
h = lambda x: P((A*P(x, e12).grade(1)*B).grade(1), e12) + P((B*P(x, e3).grade(1)*C).grade(1), e3)
F = lambda x: h(x) - l*x
(
    char_scalar(F, 2, alg, frame, frame) * char_scalar(F, 1, alg, frame2, frame2),
    char_scalar(F, 3, alg, frame3, frame3)
)

(-13.79535830507187, -13.79535830507187)

In [46]:
outermorphism(h, e1, alg), adjoint_outermorphism(h, e1, alg)

(0.763 𝐞₁ + 0.965 𝐞₂, 0.763 𝐞₁ + 0.53 𝐞₂)

In [47]:
#3.9a
adjoint_outermorphism(h, e123, alg) | e12, outermorphism(h, e12, alg).e12 * outermorphism(h, -e3, alg)

(-0.99 𝐞₃, -0.99 𝐞₃)

In [48]:
#3.9b
outermorphism(h, e123, alg) | e3, adjoint_outermorphism(h, e3, alg).e3 * adjoint_outermorphism(h, e12, alg)

(0.99 𝐞₁₂, 0.99 𝐞₁₂)

In [49]:
def separate_indices(numbers):
    real_indices = []
    conjugate_pairs_indices = []
    visited = set()  # To keep track of indices already processed

    for i, num in enumerate(numbers):
        if np.isreal(num):  # Check if the number is real
            real_indices.append(i)
        elif i not in visited:  # Check if the index is not already processed
            conjugate = np.conj(num)
            for j in range(i + 1, len(numbers)):
                if numbers[j] == conjugate:
                    conjugate_pairs_indices.append((i, j))
                    visited.add(i)
                    visited.add(j)
                    break

    return real_indices, conjugate_pairs_indices

separate_indices(eigvalues), eigvalues

(([0, 1, 2, 3], []),
 array([ 1.37557497,  0.63099487, -4.48429983, -2.65047795]))

In [50]:
# An example of non proper blade:
def linalg_eigblades(f, alg: Algebra):
    matrix = trans2matrix(f, alg)
    eigvalues, eigvecs = np.linalg.eig(matrix)
    real_indices, conjugate_pairs_indices = separate_indices(eigvalues)
    eigblades = [alg.vector(np.real(eigvecs[:, i])) for i in real_indices]
    for k, l in conjugate_pairs_indices:
        v1 = np.real(eigvecs[:,k] + eigvecs[:,l])
        v2 = np.imag(eigvecs[:,k] - eigvecs[:,l])
        eigblades.append(alg.vector(v1) ^ alg.vector(v2))
    return eigblades 

def terms_ratio(A, B: MultiVector):
    valid_keys = [k for k in B.keys() if not np.isclose(B[k], 0)]
    return np.divide([A[k] for k in valid_keys], [B[k] for k in valid_keys])

eigblades = linalg_eigblades(f, alg)

# right eigenblades but not left!
for blade in eigblades:
    print(
        terms_ratio(outermorphism(f, blade, alg), blade), 
        terms_ratio(adjoint_outermorphism(f, blade, alg), blade)
        )

[1.37557497 1.37557497 1.37557497 1.37557497] [ 2.67393444  0.2727106  -0.24321671  3.44810985]
[0.63099487 0.63099487 0.63099487 0.63099487] [ 8.15228099  0.26777628 -0.11386849  2.4921132 ]
[-4.48429983 -4.48429983 -4.48429983 -4.48429983] [-6.75705426 -4.15070262 -2.38678096 -5.40992491]
[-2.65047795 -2.65047795 -2.65047795 -2.65047795] [-43.32338826  -2.33408478  -2.55094889  -9.41694514]


In [51]:
#3.11 reciprocal for blades
def reciprocal(blades):
    I = wedge(blades)
    dualblades = []
    for k in range(len(blades)):
        sign = (-1) ** (blades[k].grades[0] * sum(blade.grades[0] for blade in blades[:k]))
        dualblades.append(sign * wedge(blades[:k] + blades[k+1:]) * I.inv())
    return dualblades

dualblades = reciprocal(eigblades)
# The reciprocal of a right eigenblade is a left eigenblade! Note that they correspond to the same eigenvalues
for blade in dualblades:
    print(
        terms_ratio(outermorphism(f, blade, alg), blade), 
        terms_ratio(adjoint_outermorphism(f, blade, alg), blade)
        )


[  1.57663867 -16.58350253  -9.64556783  -0.24099676] [1.37557497 1.37557497 1.37557497 1.37557497]
[ 1.9145999  -0.26653047  0.29975954 -0.62167964] [0.63099487 0.63099487 0.63099487 0.63099487]
[ -4.30355291  -4.99230346 -21.1834803   -4.24743471] [-4.48429983 -4.48429983 -4.48429983 -4.48429983]
[-6.73672753 -2.93918002 -1.99668152 -5.86693463] [-2.65047795 -2.65047795 -2.65047795 -2.65047795]


In [52]:
#3.12 
[eigblades[i] | dualblades[j] for i,j in [(0,0), (0,1), (1,1)]]

[1.0, 5.55e-17, 1.0]

In [53]:
# What transformation have a non-trivial proper blade?
# A invariant subspace, say e123 of e1234
# from 3.12 we know we're looking for blades equal to thier own reciprocal
# that means orthogonal to other eigenspaces

pf = lambda x: P(f(P(x, e123)), e123) + P(f(P(x, e4)), e4)
[(outermorphism(pf, blade, alg), adjoint_outermorphism(pf, blade, alg)) for blade in [e123, e4]]

[(3.18 𝐞₁₂₃, 3.18 𝐞₁₂₃), (-3.27 𝐞₄, -3.27 𝐞₄)]

In [54]:
eigblades = linalg_eigblades(pf, alg)
for E in [e123, e4]:
    for blade in eigblades: # not just vectors
        blade = (blade | E)*E
        om = outermorphism(pf, blade, alg)
        am = adjoint_outermorphism(pf, blade, alg)
        print(
            terms_ratio(om, blade), 
            terms_ratio(am, blade)
            )

[1.50706754 1.50706754 1.50706754] [ 2.37800484  0.06980381 -0.3853096 ]
[-0.83289204 -0.83289204 -0.83289204] [-7.48251265 -0.80064978 -0.20480641]
[-2.53263644 -2.53263644 -2.53263644] [ 1.72663394 -2.24905268 -2.77163649]
[] []
[] []
[] []
[] []
[-3.26974701] [-3.26974701]


In [55]:
#4.1b A matrix from a symmetric transformation is symmetric
fp = lambda x: (f(x) + adjoint_outermorphism(f, x, alg))/2
trans2matrix(fp, alg), fp(x), derivative(lambda x: x | f(x)/2, x, alg, grade=1)

(array([[ 1.25533263,  0.75996719,  0.54348343, -0.97489517],
        [ 0.75996719, -1.24707978,  0.94412357,  1.55900642],
        [ 0.54348343,  0.94412357, -1.86671379,  1.29370533],
        [-0.97489517,  1.55900642,  1.29370533, -3.26974701]]),
 1.53 𝐞₁ + 1.42 𝐞₂ + 0.655 𝐞₃ + -0.942 𝐞₄,
 1.53 𝐞₁ + 1.42 𝐞₂ + 0.655 𝐞₃ + -0.942 𝐞₄)

In [56]:
# a curl is a part of a derivative
derivative(f, x, alg, grade=1).grade(2), curl(f, x, alg, grade=1)

(0.75 𝐞₁₂ + 0.965 𝐞₁₃ + 0.398 𝐞₁₄ + -0.193 𝐞₂₃ + -0.0751 𝐞₂₄ + -1.16 𝐞₃₄,
 0.75 𝐞₁₂ + 0.965 𝐞₁₃ + 0.398 𝐞₁₄ + -0.193 𝐞₂₃ + -0.0751 𝐞₂₄ + -1.16 𝐞₃₄)

In [57]:
#4.1c a skew transfromation leads to a skew matrix
fm = lambda x: (f(x) - adjoint_outermorphism(f, x, alg))/2
trans2matrix(fm, alg), fm(x), x | derivative(f, x, alg, grade=1).grade(2)/2

(array([[-1.11022302e-16, -3.74882257e-01, -4.82345292e-01,
         -1.99217107e-01],
        [ 3.74882257e-01, -3.33066907e-16,  9.66419740e-02,
          3.75671771e-02],
        [ 4.82345292e-01, -9.66419740e-02, -1.11022302e-16,
          5.80132664e-01],
        [ 1.99217107e-01, -3.75671771e-02, -5.80132664e-01,
          2.22044605e-16]]),
 -0.657 𝐞₁ + 0.45 𝐞₂ + 0.732 𝐞₃ + -0.199 𝐞₄,
 -0.657 𝐞₁ + 0.45 𝐞₂ + 0.732 𝐞₃ + -0.199 𝐞₄)

In [58]:
#4.2 fp curl free
a | fp(x), x | fp(a), curl(fp, x, alg, grade=1)

(1.2,
 1.2,
 -1.11e-10 𝐞₁₂ + 3.33e-10 𝐞₁₃ + 3.89e-10 𝐞₁₄ + 2.22e-10 𝐞₂₃ + -5.55e-11 𝐞₂₄ + 5.55e-11 𝐞₃₄)

In [59]:
#4.3 spectral decomposition of the symmetric part
matrix = trans2matrix(fp, alg)
eigvalues, eigvecs = np.linalg.eigh(matrix)
eigvecs = [alg.vector(v) for v in eigvecs.transpose()]
mf = lambda x: sum(e * P(x, v) for e, v in zip(eigvalues, eigvecs))
mf(x), fp(x)

(1.53 𝐞₁ + 1.42 𝐞₂ + 0.655 𝐞₃ + -0.942 𝐞₄,
 1.53 𝐞₁ + 1.42 𝐞₂ + 0.655 𝐞₃ + -0.942 𝐞₄)

In [60]:
# QR factorization
transframe = [fp(e) for e in alg.frame] #another basis
gs_trans = gram_schmidt(transframe)
gs_trans2 = gram_schmidt([fp(v / norm(v)) for v in gs_trans[::-1]])[::-1]
gs_trans2


[-34.4 𝐞₁ + -4.01e-15 𝐞₂ + -3.87e-15 𝐞₃ + -2.46e-15 𝐞₄,
 9.56e-16 𝐞₁ + -8.22 𝐞₂ + -1.94e-15 𝐞₃ + -2.57e-15 𝐞₄,
 1.66e-16 𝐞₁ + 3.49e-16 𝐞₂ + -1.48 𝐞₃ + 2.28e-16 𝐞₄,
 5.55e-17 𝐞₁ + 2.43e-16 𝐞₂ + -1.2e-16 𝐞₃ + -0.776 𝐞₄]

In [61]:
# A GA version of eigenblade-solver given the eigenvalues
def null_space(f, alg, tol=1e-5, frame=None):
    if not frame:
        frame = alg.frame
    null_sp = wedge(frame)
    images = [f(a) for a in frame]
    for v in images:
        contracted = v | null_sp
        if normsq(contracted) >= tol:
            null_sp = contracted
    return null_sp

def eigenblades_from_values(f, eigvalues, alg, frame=None):
    eigblades = []
    for e in eigvalues:
        null_f = lambda x: f(x) - e*x
        eigblades.append(null_space(null_f, alg, frame=frame))
    return eigblades

In [62]:
# One thing to note for a symmetric transformation:
# the characteristic multivectors are of scalar values
# So all subspaces are invariant under the transfromation.
[char_multi_linear(fp, r, alg) for r in range(5)]

[1,
 -5.13 + 1.11e-16 𝐞₁₃ + 2.22e-16 𝐞₂₃,
 -2.32 + -8.88e-16 𝐞₁₃ + -2.22e-16 𝐞₁₄ + -8.88e-16 𝐞₂₃ + -8.88e-16 𝐞₂₄,
 19.5 + 8.88e-16 𝐞₁₂ + -8.88e-16 𝐞₂₃,
 7.96]

In [63]:
np.roots(char_poly_coefs(fp, alg)), eigvalues

(array([-4.6432703 , -2.54621991,  1.65396858,  0.40731369]),
 array([-4.6432703 , -2.54621991,  0.40731369,  1.65396858]))

In [64]:
eigblades = eigenblades_from_values(fp, eigvalues, alg)
[e/norm(e) for e in eigblades], eigvecs

([0.216 𝐞₁ + -0.351 𝐞₂ + -0.32 𝐞₃ + 0.853 𝐞₄,
  -0.0208 𝐞₁ + 0.621 𝐞₂ + -0.783 𝐞₃ + -0.0334 𝐞₄,
  -0.289 𝐞₁ + 0.637 𝐞₂ + 0.491 𝐞₃ + 0.519 𝐞₄,
  0.933 𝐞₁ + 0.292 𝐞₂ + 0.209 𝐞₃ + -0.0374 𝐞₄],
 [0.216 𝐞₁ + -0.351 𝐞₂ + -0.32 𝐞₃ + 0.853 𝐞₄,
  0.0208 𝐞₁ + -0.621 𝐞₂ + 0.783 𝐞₃ + 0.0334 𝐞₄,
  0.289 𝐞₁ + -0.637 𝐞₂ + -0.491 𝐞₃ + -0.519 𝐞₄,
  0.933 𝐞₁ + 0.292 𝐞₂ + 0.209 𝐞₃ + -0.0374 𝐞₄])

In [65]:
#4.4 modify mf so to have a repeat eigenvalue (degeneracy)
eigvalues[1] = eigvalues[0]
mf = lambda x: sum(e * P(x, v) for e, v in zip(eigvalues, eigvecs))
for blade in [eigvecs[0], wedge(eigvecs[:2])]:
    of = outermorphism(mf, blade, alg)
    print(terms_ratio(of, blade))

[-4.6432703 -4.6432703 -4.6432703 -4.6432703]
[21.55995904 21.55995904 21.55995904 21.55995904 21.55995904 21.55995904]


In [66]:
eigblades = eigenblades_from_values(mf, set(eigvalues), alg)
[e/norm(e) for e in eigblades], [wedge(eigvecs[:2])] + eigvecs[2:]

([-0.289 𝐞₁ + 0.637 𝐞₂ + 0.491 𝐞₃ + 0.519 𝐞₄,
  0.933 𝐞₁ + 0.292 𝐞₂ + 0.209 𝐞₃ + -0.0374 𝐞₄,
  0.127 𝐞₁₂ + -0.176 𝐞₁₃ + 0.0106 𝐞₁₄ + 0.474 𝐞₂₃ + -0.518 𝐞₂₄ + 0.678 𝐞₃₄],
 [-0.127 𝐞₁₂ + 0.176 𝐞₁₃ + -0.0106 𝐞₁₄ + -0.474 𝐞₂₃ + 0.518 𝐞₂₄ + -0.678 𝐞₃₄,
  0.289 𝐞₁ + -0.637 𝐞₂ + -0.491 𝐞₃ + -0.519 𝐞₄,
  0.933 𝐞₁ + 0.292 𝐞₂ + 0.209 𝐞₃ + -0.0374 𝐞₄])

In [67]:
#4.6 a reflection is an orthogonal symmetric transformation 
A3 = random_r_blade(3, alg).normalized()
matrix = trans2matrix(lambda x: (-1)**3*(A3.sw(x)), alg)
# a symmetric matrix, orthogonal, unit eigvals
matrix, [np.dot(matrix[i], matrix[j]) for i,j in [(1,1), (1,2), (2,3), (3,3)]], np.linalg.eigvals(matrix)

(array([[-0.37441358, -0.6857989 , -0.2404855 ,  0.57590022],
        [-0.6857989 , -0.24819319,  0.26363214, -0.63133042],
        [-0.2404855 ,  0.26363214, -0.9075535 , -0.22138532],
        [ 0.57590022, -0.63133042, -0.22138532, -0.46983973]]),
 [1.0000000000000007, 0.0, 2.7755575615628914e-17, 1.0000000000000007],
 array([-1.+0.00000000e+00j,  1.+0.00000000e+00j, -1.+1.57009246e-16j,
        -1.-1.57009246e-16j]))

In [68]:
#4.6 Find blade A from an orthogonal-symmetric-transfromation?
ot = matrix2trans(matrix, alg)
eigvalues, eigvecs = np.linalg.eig(matrix)
eigvecs = [alg.vector(v) for v in eigvecs.transpose()]
A3_ = wedge([v for v, e in zip(eigvecs, eigvalues) if np.isclose(e, -1)])
(-1)**3*(A3.sw(x)), ot(x)

(-0.64 𝐞₁ + -1.01 𝐞₂ + -0.771 𝐞₃ + -0.237 𝐞₄,
 -0.64 𝐞₁ + -1.01 𝐞₂ + -0.771 𝐞₃ + -0.237 𝐞₄)

In [69]:
#4.7 Not true for whole space, but the proper ones
# In matrix terms: diagonal parts commute
eigblades = linalg_eigblades(pf, alg)
eigblades.append(e123)
eigblades, [max_diff(pf(blade.sw(x)), blade.sw(pf(x))) for blade in eigblades]

([0.805 𝐞₁ + 0.469 𝐞₂ + 0.363 𝐞₃,
  0.17 𝐞₁ + -0.838 𝐞₂ + -0.519 𝐞₃,
  0.0547 𝐞₁ + -0.657 𝐞₂ + 0.752 𝐞₃,
  1.0 𝐞₄,
  1 𝐞₁₂₃],
 [0.28843253507106703, 2.2344130042726107, 0.007435051305568652, 0.0, 0.0])

In [70]:
#4.8 skew transformation -> skew bilinear form -> bivector
fm_biform = lambda vectors: vectors[0] | fm(vectors[1])
F = simplicial_derivative(fm_biform, np.zeros(2), alg)
a | fm(x), a| (x | F)

(-0.305, -0.305)

In [71]:
#4.9 so fm(x) is always orthogonal to x
fm(x), x|F, fm(x) | x

(-0.657 𝐞₁ + 0.45 𝐞₂ + 0.732 𝐞₃ + -0.199 𝐞₄,
 -0.657 𝐞₁ + 0.45 𝐞₂ + 0.732 𝐞₃ + -0.199 𝐞₄,
 1.25e-16)

In [72]:
#4.10 F is a bivector
derivative(fm, x, alg, grade=1), curl(fm, x, alg, grade=1), curl(f, x, alg, grade=1), 2*F

(-1.67e-10 + 0.75 𝐞₁₂ + 0.965 𝐞₁₃ + 0.398 𝐞₁₄ + -0.193 𝐞₂₃ + -0.0751 𝐞₂₄ + -1.16 𝐞₃₄,
 0.75 𝐞₁₂ + 0.965 𝐞₁₃ + 0.398 𝐞₁₄ + -0.193 𝐞₂₃ + -0.0751 𝐞₂₄ + -1.16 𝐞₃₄,
 0.75 𝐞₁₂ + 0.965 𝐞₁₃ + 0.398 𝐞₁₄ + -0.193 𝐞₂₃ + -0.0751 𝐞₂₄ + -1.16 𝐞₃₄,
 0.75 𝐞₁₂ + 0.965 𝐞₁₃ + 0.398 𝐞₁₄ + -0.193 𝐞₂₃ + -0.0751 𝐞₂₄ + -1.16 𝐞₃₄)

In [73]:
#4.11 bivector split commutes, think about a screw motion splits into translation and rotation
e13*e24, e24*e13

(-1 𝐞₁₂₃₄, -1 𝐞₁₂₃₄)

In [74]:
#4.13 How to factor F?
def wedge_power(A, r):
    if r == 0:
        return 1
    result = A
    for _ in range(r-1):
        result = A ^ result
    return result

Ck = [(1/factorial(r))*wedge_power(F, r) for r in range(3)]
Ck, [char_multi_linear(fm, r, alg) for r in range(1, 5)] # these are full story, but we just take a fraction

([1.0,
  0.375 𝐞₁₂ + 0.482 𝐞₁₃ + 0.199 𝐞₁₄ + -0.0966 𝐞₂₃ + -0.0376 𝐞₂₄ + -0.58 𝐞₃₄,
  -0.219 𝐞₁₂₃₄],
 [-3.33e-16 + 0.75 𝐞₁₂ + 0.965 𝐞₁₃ + 0.398 𝐞₁₄ + -0.193 𝐞₂₃ + -0.0751 𝐞₂₄ + -1.16 𝐞₃₄,
  0.76 + 1.58e-16 𝐞₁₂ + -5.55e-17 𝐞₁₃ + -3.33e-16 𝐞₁₄ + -1.46e-16 𝐞₂₃ + -1.25e-16 𝐞₂₄ + 4.65e-16 𝐞₃₄ + -0.874 𝐞₁₂₃₄,
  -1.48e-16 + 0.254 𝐞₁₂ + -0.0164 𝐞₁₃ + 0.0423 𝐞₁₄ + -0.0871 𝐞₂₃ + 0.211 𝐞₂₄ + -0.164 𝐞₃₄,
  0.0478])

In [75]:
[alg.ip(Ck[i-1], Ck[i]) for i in range(1, len(Ck))]

[0.375 𝐞₁₂ + 0.482 𝐞₁₃ + 0.199 𝐞₁₄ + -0.0966 𝐞₂₃ + -0.0376 𝐞₂₄ + -0.58 𝐞₃₄,
 -0.127 𝐞₁₂ + 0.00821 𝐞₁₃ + -0.0211 𝐞₁₄ + 0.0436 𝐞₂₃ + -0.105 𝐞₂₄ + 0.082 𝐞₃₄]

In [76]:
set.union(*(set(ck.keys()) for ck in Ck if isinstance(ck, MultiVector)))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15}

In [77]:
x|F, fm(x), x|F|F, fm(fm(x))

(-0.657 𝐞₁ + 0.45 𝐞₂ + 0.732 𝐞₃ + -0.199 𝐞₄,
 -0.657 𝐞₁ + 0.45 𝐞₂ + 0.732 𝐞₃ + -0.199 𝐞₄,
 -0.482 𝐞₁ + -0.183 𝐞₂ + -0.476 𝐞₃ + -0.572 𝐞₄,
 -0.482 𝐞₁ + -0.183 𝐞₂ + -0.476 𝐞₃ + -0.572 𝐞₄)

In [78]:
#4.14 4.18 Why square of eigen values? 
# The odd grade characteristic multivectors have zero scalar
# A correction on Lasenby's paper, not square of f, but the even outermorphism f_(x, y)
Ck2 = [(-1)**(2 - i) * alg.sp(Ck[i], Ck[i])[0] for i in range(3)]
char_poly = char_poly_coefs(fm, alg)
Lk = np.roots(Ck2)

Ck2, char_poly, Lk, np.sqrt(-np.array(Lk)), np.roots(char_poly), char_poly_coefs(lambda x: fm(fm(x)), alg)

([1.0, 0.7601860138604837, 0.047792004728342836],
 [1,
  3.3306690738754696e-16,
  0.7601860138604839,
  1.478851763270228e-16,
  0.04779200472834285],
 array([-0.69102498, -0.06916104]),
 array([0.83127912, 0.26298486]),
 array([-6.61498202e-17+0.83127912j, -6.61498202e-17-0.83127912j,
        -1.00383634e-16+0.26298486j, -1.00383634e-16-0.26298486j]),
 [1,
  1.5203720277209678,
  0.6734667851257777,
  0.07266162713768079,
  0.0022840757159539515])

In [79]:
Ck[1], Ck[1] | Ck[2]

(0.375 𝐞₁₂ + 0.482 𝐞₁₃ + 0.199 𝐞₁₄ + -0.0966 𝐞₂₃ + -0.0376 𝐞₂₄ + -0.58 𝐞₃₄,
 -0.127 𝐞₁₂ + 0.00821 𝐞₁₃ + -0.0211 𝐞₁₄ + 0.0436 𝐞₂₃ + -0.105 𝐞₂₄ + 0.082 𝐞₃₄)

In [80]:
#4.15
mv_map1 = {}
mv_map2 = {}
ck1, ck2 = Ck[1], Ck[1] | Ck[2]
for key in ck2.keys():
    if np.isclose(ck1[key],0) and np.isclose(ck2[key],0):
        continue
    ans = np.linalg.solve(np.array([[1, 1], Lk]), [ck1[key], ck2[key]])
    mv_map1[key], mv_map2[key] = ans
F1 = alg.multivector(mv_map1)
F2 = alg.multivector(mv_map2)
F1 + F2, F, F1*F2, F2*F1, F**2, F1**2, F2**2


(0.375 𝐞₁₂ + 0.482 𝐞₁₃ + 0.199 𝐞₁₄ + -0.0966 𝐞₂₃ + -0.0376 𝐞₂₄ + -0.58 𝐞₃₄,
 0.375 𝐞₁₂ + 0.482 𝐞₁₃ + 0.199 𝐞₁₄ + -0.0966 𝐞₂₃ + -0.0376 𝐞₂₄ + -0.58 𝐞₃₄,
 -7.63e-17 + 6.94e-18 𝐞₁₂ + 8.67e-18 𝐞₁₃ + -4.16e-17 𝐞₁₄ + 4.68e-17 𝐞₂₃ + 1.21e-17 𝐞₂₄ + -9.54e-18 𝐞₃₄ + -0.219 𝐞₁₂₃₄,
 -7.63e-17 + -5.64e-18 𝐞₁₂ + -8.67e-18 𝐞₁₃ + 4.16e-17 𝐞₁₄ + -4.16e-17 𝐞₂₃ + -1.39e-17 𝐞₂₄ + 1.04e-17 𝐞₃₄ + -0.219 𝐞₁₂₃₄,
 -0.76 + -1.73e-18 𝐞₁₂ + -0.437 𝐞₁₂₃₄,
 -0.0692 + -4.34e-19 𝐞₁₂ + -2.17e-19 𝐞₁₃ + -1.73e-18 𝐞₁₄ + -3.64e-17 𝐞₁₂₃₄,
 -0.691 + -1.25e-16 𝐞₁₂₃₄)

In [81]:
#4.16
def normsq(A):
    return A.reverse().sp(A)[0]
(
    [(-normsq(F) - np.sqrt((normsq(F)**2 - normsq(F ^ F))))/2,
    (-normsq(F) + np.sqrt((normsq(F)**2 - normsq(F ^ F))))/2],
    Lk
)


([-0.6910249756773137, -0.06916103818316999],
 array([-0.69102498, -0.06916104]))

In [82]:
#4.17
F2_ = F / (1 + (1/2/Lk[0])*(F ^ F))
F1_ = F / (1 + (1/2/Lk[1])*(F ^ F))
F1_, F1, F2_, F2

(0.162 𝐞₁₂ + -0.0669 𝐞₁₃ + 0.0118 𝐞₁₄ + -0.0593 𝐞₂₃ + 0.174 𝐞₂₄ + -0.0673 𝐞₃₄,
 0.162 𝐞₁₂ + -0.0669 𝐞₁₃ + 0.0118 𝐞₁₄ + -0.0593 𝐞₂₃ + 0.174 𝐞₂₄ + -0.0673 𝐞₃₄,
 0.213 𝐞₁₂ + 0.549 𝐞₁₃ + 0.187 𝐞₁₄ + -0.0374 𝐞₂₃ + -0.211 𝐞₂₄ + -0.513 𝐞₃₄,
 0.213 𝐞₁₂ + 0.549 𝐞₁₃ + 0.187 𝐞₁₄ + -0.0374 𝐞₂₃ + -0.211 𝐞₂₄ + -0.513 𝐞₃₄)

In [83]:
def bivector_split(F, alg: Algebra):
    m = alg.d // 2
    if m <= 1:
        return F
    Ck = [(1/factorial(r))*wedge_power(F, r) for r in range(m+1)]
    Ck2 = [(-1)**(m - i) * alg.sp(Ck[i], Ck[i])[0] for i in range(m+1)]
    Lk = np.roots(Ck2)
    mv_map_list = [{} for _ in range(m)]
    ck_inner_list = [alg.ip(Ck[i-1], Ck[i]) for i in range(1, m+1)]
    mv_keys = set.union(*(set(ck.keys()) for ck in ck_inner_list if isinstance(ck, MultiVector)))
    linear_eq_matrix = np.array([[np.prod(Lk[i:i+k]) for i in range(m)] for k in range(m)])
    inv_matrix = np.linalg.inv(linear_eq_matrix)
    for key in mv_keys:
        if all([np.isclose(ck[key],0) for ck in ck_inner_list]):
            continue
        ans = np.dot(inv_matrix, [ck[key] for ck in ck_inner_list])
        for i, v in enumerate(ans):
            mv_map_list[i][key] = v
    return [alg.multivector(mv_map) for mv_map in mv_map_list]

In [84]:
# From Graded Symmetry Groups: Plane and Simple, 
# instead of solving linear equations, the solutions are given algebraically
# But we need an GA inverse for even algebra
def bivector_split_(F, alg: Algebra):  
    m = alg.d // 2
    if m <= 1:
        return F
    Ck = [(1/factorial(r))*wedge_power(F, r) for r in range(m+1)]
    Ck2 = [(-1)**(m - i) * alg.sp(Ck[i], Ck[i])[0] for i in range(m+1)]
    Lk = np.roots(Ck2)
    blades = []
    for l in Lk:
        even = sum(l ** r * c for r, c in enumerate(Ck[-1::-2]))
        odd = sum(l ** r * c for r, c in enumerate(Ck[-2::-2]))
        blades.append(even / odd) # FIXME: the inverse usually fails
    return blades

In [85]:
# What if we use the above function to split a bivector that's almost a blade?
# The result blades are R itself and its dual with a small magnitude
# Should assert not simple before process?

R = random_r_blade(2, alg) + 1e-12 * A # a blade with a small perturbation
blades = bivector_split(R, alg)
R**2, [terms_ratio(b, r) for r in [R, R.dual()] for b in blades]

(-2.08 + -1.85e-12 𝐞₁ + -2.65e-13 𝐞₂ + 1.8e-12 𝐞₃ + 5.39e-13 𝐞₄ + -1.05e-12 𝐞₁₂ + 6.15e-13 𝐞₁₃ + 2.01e-13 𝐞₁₄ + -3.28e-13 𝐞₂₃ + 4.44e-13 𝐞₂₄ + 1.02e-12 𝐞₃₄ + 4.11e-14 𝐞₁₂₃ + 7.36e-13 𝐞₁₂₄ + 4.05e-13 𝐞₁₃₄ + -3.06e-13 𝐞₂₃₄ + 1.1e-12 𝐞₁₂₃₄,
 [array([-3.76432384e-13,  9.40734110e-13,  1.19853776e-13,  5.85897843e-13,
          7.46460324e-14, -1.86546302e-13]),
  array([1., 1., 1., 1., 1., 1.]),
  array([-2.6499447e-13, -2.6499447e-13, -2.6499447e-13, -2.6499447e-13,
         -2.6499447e-13, -2.6499447e-13]),
  array([ 0.703963  , -0.28168902, -2.2109814 , -0.45228784, -3.55001413,
          1.42052921])])

In [86]:
# double skew is symmetric
fm2 = lambda x: fm(fm(x))
curl(fm2, x, alg, h=1e-2, grade=1)

2.78e-15 𝐞₁₂ + 2.78e-15 𝐞₁₃ + 2.5e-14 𝐞₁₄ + 1.11e-14 𝐞₂₃ + -8.33e-15 𝐞₂₄

In [87]:
#4.19
v = P(e1, F1)
fm2(v), (F1 ** 2) * v

(-0.0309 𝐞₁ + -0.00602 𝐞₂ + -0.00882 𝐞₃ + 0.0327 𝐞₄,
 -0.0309 𝐞₁ + -0.00602 𝐞₂ + -0.00882 𝐞₃ + 0.0327 𝐞₄ + 1.72e-17 𝐞₁₂₃ + 5e-18 𝐞₁₂₄ + -2.85e-18 𝐞₁₃₄ + 1.63e-17 𝐞₂₃₄)

In [88]:
#4.20 
F1_eigvecs= [v.normalized() for v in gram_schmidt(blade_split(F1, alg))]
F1, np.sqrt(normsq(F1)) * wedge(F1_eigvecs)


(0.162 𝐞₁₂ + -0.0669 𝐞₁₃ + 0.0118 𝐞₁₄ + -0.0593 𝐞₂₃ + 0.174 𝐞₂₄ + -0.0673 𝐞₃₄,
 0.162 𝐞₁₂ + -0.0669 𝐞₁₃ + 0.0118 𝐞₁₄ + -0.0593 𝐞₂₃ + 0.174 𝐞₂₄ + -0.0673 𝐞₃₄)

In [89]:
# If Lk[0] == Lk[1], the linear equation has no unique solution in 4.15
# nonunique bivector split? We should find another split of e12 + e34. 
t = 1.1
R = blade_exp(t*e13) * blade_exp(t*e24)
a,b,c,d = [R.sw(v) for v in alg.frame]
a,b,c,d, a*b+c*d, a*b, c*d

(-0.589 𝐞₁ + -0.808 𝐞₃,
 -0.589 𝐞₂ + -0.808 𝐞₄,
 0.808 𝐞₁ + -0.589 𝐞₃,
 0.808 𝐞₂ + -0.589 𝐞₄,
 1.0 𝐞₁₂ + 1.0 𝐞₃₄,
 0.346 𝐞₁₂ + 0.476 𝐞₁₄ + -0.476 𝐞₂₃ + 0.654 𝐞₃₄,
 0.654 𝐞₁₂ + -0.476 𝐞₁₄ + 0.476 𝐞₂₃ + 0.346 𝐞₃₄)

In [90]:
R.sw(e12 + e34), (e12+e34) * (R**2), R**2

(1.0 𝐞₁₂ + 1.0 𝐞₃₄,
 1.0 𝐞₁₂ + 1.0 𝐞₃₄,
 0.346 + -0.476 𝐞₁₃ + -0.476 𝐞₂₄ + -0.654 𝐞₁₂₃₄)

In [91]:
#4.21
B2 = B.grade(2)
# fm = lambda x: f(x) - adjoint_outermorphism(f, x, alg)
# F = curl(fm, x, alg, grade=1) / 2
outermorphism(fm, B2, alg), (0.5 * (B2 | (F^F))) - ((B2|F) * F)

(0.362 𝐞₁₂ + 0.301 𝐞₁₃ + 0.191 𝐞₁₄ + 0.138 𝐞₂₃ + -0.233 𝐞₂₄ + -0.197 𝐞₃₄,
 0.362 𝐞₁₂ + 0.301 𝐞₁₃ + 0.191 𝐞₁₄ + 0.138 𝐞₂₃ + -0.233 𝐞₂₄ + -0.197 𝐞₃₄)

In [92]:
# What is a symmetric bivector function?
# I guess it means:
outermorphism(fm, B2, alg), adjoint_outermorphism(fm, B2, alg)

(0.362 𝐞₁₂ + 0.301 𝐞₁₃ + 0.191 𝐞₁₄ + 0.138 𝐞₂₃ + -0.233 𝐞₂₄ + -0.197 𝐞₃₄,
 0.362 𝐞₁₂ + 0.301 𝐞₁₃ + 0.191 𝐞₁₄ + 0.138 𝐞₂₃ + -0.233 𝐞₂₄ + -0.197 𝐞₃₄)

In [93]:
#4.23 The bivector split solves the eigenbivector problem of a skew transformation
outermorphism(fm, F1, alg), normsq(F1)*F1

(0.0112 𝐞₁₂ + -0.00462 𝐞₁₃ + 0.000817 𝐞₁₄ + -0.0041 𝐞₂₃ + 0.012 𝐞₂₄ + -0.00465 𝐞₃₄,
 0.0112 𝐞₁₂ + -0.00462 𝐞₁₃ + 0.000817 𝐞₁₄ + -0.0041 𝐞₂₃ + 0.012 𝐞₂₄ + -0.00465 𝐞₃₄)

In [94]:
#5.1 normal transformation
af = lambda x: adjoint_outermorphism(f, x, alg)

fp(fm(x)), fm(fp(x)), af(f(x)), f(af(x)) # not normal

(0.11 𝐞₁ + -0.68 𝐞₂ + -1.56 𝐞₃ + 2.94 𝐞₄,
 -0.659 𝐞₁ + 0.6 𝐞₂ + 0.0522 𝐞₃ + -0.129 𝐞₄,
 5.52 𝐞₁ + -2.55 𝐞₂ + -1.41 𝐞₃ + 8.29 𝐞₄,
 3.98 𝐞₁ + 0.00511 𝐞₂ + 1.81 𝐞₃ + 2.15 𝐞₄)

In [95]:
M = blade_exp(random_r_blade(2, alg))
Mdual = blade_split(M.grade(2).dual(), alg)
rigid = lambda x: 1.1 *  M.sw(x) - 4.4 * P(x, Mdual[0]) + 3.3 * P(x, Mdual[1])
adrigid = lambda x: adjoint_outermorphism(rigid, x, alg)
adrigid(rigid(x)), rigid(adrigid(x))

(9.42 𝐞₁ + -3.1 𝐞₂ + 0.051 𝐞₃ + -0.109 𝐞₄,
 9.42 𝐞₁ + -3.1 𝐞₂ + 0.051 𝐞₃ + -0.109 𝐞₄)

In [96]:
rigid(x), 3.3 * M**2 * P(x, M.grade(2).normalized()) + 3.3 * P(x, M.grade(2).normalized().dual()) - 4.4 * P(x, Mdual[0]) + 5.5 * P(x, Mdual[1])

(1.97 𝐞₁ + -1.4 𝐞₂ + 0.972 𝐞₃ + 0.662 𝐞₄,
 3.97 𝐞₁ + -3.24 𝐞₂ + 2.68 𝐞₃ + 2.59 𝐞₄ + 1.66e-18 𝐞₁₂₃ + -1.99e-18 𝐞₁₂₄ + 1.41e-16 𝐞₁₃₄ + 1.65e-16 𝐞₂₃₄)

In [97]:
#5.2
def norm(A):
    return np.sqrt(normsq(A))
b1, b2 = norm(F1), norm(F2)
i1, i2 = F1/b1, F2/b2

F, b1*i1 + b2*i2, outermorphism(fm, i1, alg), b1**2*i1

(0.375 𝐞₁₂ + 0.482 𝐞₁₃ + 0.199 𝐞₁₄ + -0.0966 𝐞₂₃ + -0.0376 𝐞₂₄ + -0.58 𝐞₃₄,
 0.375 𝐞₁₂ + 0.482 𝐞₁₃ + 0.199 𝐞₁₄ + -0.0966 𝐞₂₃ + -0.0376 𝐞₂₄ + -0.58 𝐞₃₄,
 0.0427 𝐞₁₂ + -0.0176 𝐞₁₃ + 0.00311 𝐞₁₄ + -0.0156 𝐞₂₃ + 0.0457 𝐞₂₄ + -0.0177 𝐞₃₄,
 0.0427 𝐞₁₂ + -0.0176 𝐞₁₃ + 0.00311 𝐞₁₄ + -0.0156 𝐞₂₃ + 0.0457 𝐞₂₄ + -0.0177 𝐞₃₄)

In [98]:
# 5.3
terms_ratio(outermorphism(fp, i1, alg), i1) # not normal

array([ -3.54057242, -12.67784067,   7.86852222,   9.14237702,
         3.84897466,  15.9711852 ])

In [99]:
# 5.3
def sym_part(f, alg):
    return lambda x: (f(x) + adjoint_outermorphism(f, x, alg))/2

def skew_part(f, alg):
    return lambda x: (f(x) - adjoint_outermorphism(f, x, alg))/2

R = curl(rigid, x, alg, grade=1)/2
skew_blades = bivector_split(R, alg)
skew_eigs = [norm(b) for b in skew_blades]
skew_blades = [b / e for b,e in zip(skew_blades, skew_eigs) if e > 1e-6]
skew_eigs = [e for e in skew_eigs if e > 1e-6]
rigid_p = sym_part(rigid, alg)
[terms_ratio(outermorphism(rigid_p, i, alg), i) for i in skew_blades]

[array([0.23735426, 0.23735426, 0.23735426, 0.23735426, 0.23735426,
        0.23735426])]

In [100]:
#5.4
rand_vec = random_r_vectors(1, alg)[0]
sym_vecs = [(rand_vec | i).normalized() for i in skew_blades]
other_sym_space = wedge(skew_blades).dual()
sym_eigs = [terms_ratio(rigid_p(a), a)[0] for a in sym_vecs]

sym_eigs, np.square(sym_eigs), [terms_ratio(outermorphism(rigid_p, i, alg), i)[0] for i in skew_blades]


([0.487190171817975], array([0.23735426]), [0.2373542644461067])

In [101]:
#5.5a
m = len(skew_blades)
rigid_eigs = [sym_eigs[k] + skew_blades[k] * skew_eigs[k] for k in range(m)]
[rigid(a) for a in sym_vecs], [a*l for a, l in zip(sym_vecs,rigid_eigs)], [l.reverse()*a for a, l in zip(sym_vecs,rigid_eigs)]

([-0.43 𝐞₁ + -0.722 𝐞₂ + -0.548 𝐞₃ + -0.451 𝐞₄],
 [-0.43 𝐞₁ + -0.722 𝐞₂ + -0.548 𝐞₃ + -0.451 𝐞₄ + -4.86e-17 𝐞₁₂₃ + 6.94e-18 𝐞₁₂₄ + -1.39e-17 𝐞₁₃₄],
 [-0.43 𝐞₁ + -0.722 𝐞₂ + -0.548 𝐞₃ + -0.451 𝐞₄ + 5.55e-17 𝐞₁₂₃ + 1.73e-17 𝐞₁₃₄ + 1.39e-17 𝐞₂₃₄])

In [102]:
#5.5b
[adjoint_outermorphism(rigid, a, alg) for a in sym_vecs], [l*a for a, l in zip(sym_vecs,rigid_eigs)], [a*l.reverse() for a, l in zip(sym_vecs,rigid_eigs)]

([0.0921 𝐞₁ + -0.103 𝐞₂ + 0.768 𝐞₃ + 0.776 𝐞₄],
 [0.0921 𝐞₁ + -0.103 𝐞₂ + 0.768 𝐞₃ + 0.776 𝐞₄ + -5.55e-17 𝐞₁₂₃ + -1.73e-17 𝐞₁₃₄ + -1.39e-17 𝐞₂₃₄],
 [0.0921 𝐞₁ + -0.103 𝐞₂ + 0.768 𝐞₃ + 0.776 𝐞₄ + 4.86e-17 𝐞₁₂₃ + -6.94e-18 𝐞₁₂₄ + 1.39e-17 𝐞₁₃₄])

In [103]:
[norm(l) for l in rigid_eigs], [np.sqrt(sym_eigs[k] ** 2 + skew_eigs[k] ** 2) for k in range(m)]

([1.0999999999964827], [1.0999999999964827])

In [104]:
#5.6
rho = [norm(l) for l in rigid_eigs] # dilation
theta = [np.arccos(sym_eigs[k] / rho[k]) for k in range(m)] # rotation angle

rigid_eigs, [rho[k] * blade_exp(theta[k] * skew_blades[k]) for k in range(m)] # skew_blades gives rotation plane

([0.487 + -0.114 𝐞₁₂ + 0.287 𝐞₁₃ + 0.3 𝐞₁₄ + 0.627 𝐞₂₃ + 0.623 𝐞₂₄ + 0.0802 𝐞₃₄],
 [0.487 + -0.114 𝐞₁₂ + 0.287 𝐞₁₃ + 0.3 𝐞₁₄ + 0.627 𝐞₂₃ + 0.623 𝐞₂₄ + 0.0802 𝐞₃₄])

In [105]:
np.sqrt(1 - (skew_eigs[0]/rho[0])**2) * rho[0]

0.4871901718179753

In [106]:
# spectral decomposition of symmetric part in other_sym_space
other_sym_frame = blade_split(other_sym_space, alg)
other_sym_eigs = np.roots(char_poly_coefs(rigid_p, alg, other_sym_frame, reciprocal(other_sym_frame)))
other_sym_eigs

array([ 4.35528833, -3.25528833])

In [107]:
#5.9 spectral decomposition of a normal transformation
sym_vecs_dual = [a|i for a, i in zip(sym_vecs, skew_blades)]
other_sym_blades = eigenblades_from_values(rigid_p, other_sym_eigs, alg, other_sym_frame)

rigid(x), sum([P(x, a) * l for a, l in zip(sym_vecs + sym_vecs_dual + other_sym_blades, rigid_eigs * 2 + list(other_sym_eigs))])

(1.97 𝐞₁ + -1.4 𝐞₂ + 0.972 𝐞₃ + 0.662 𝐞₄,
 1.97 𝐞₁ + -1.4 𝐞₂ + 0.972 𝐞₃ + 0.662 𝐞₄ + 2.08e-17 𝐞₁₂₃ + -1.39e-17 𝐞₁₂₄ + -2.08e-17 𝐞₁₃₄ + -4.16e-17 𝐞₂₃₄)

In [108]:
#5.17
sym_eigs, sym_vecs = np.linalg.eigh(trans2matrix(rigid_p, alg))
sym_vecs = [alg.vector(v) for v in sym_vecs.transpose()]
sym_eigs, sym_vecs

(array([-3.25528833,  0.48719017,  0.48719017,  4.35528833]),
 [0.0792 𝐞₁ + 0.0531 𝐞₂ + -0.709 𝐞₃ + 0.699 𝐞₄,
  0.4 𝐞₁ + 0.898 𝐞₂ + -0.0599 𝐞₃ + -0.174 𝐞₄,
  0.174 𝐞₁ + 0.102 𝐞₂ + 0.701 𝐞₃ + 0.684 𝐞₄,
  -0.896 𝐞₁ + 0.425 𝐞₂ + 0.0468 𝐞₃ + 0.117 𝐞₄])

In [109]:
# In my case, all vectors are both eigenvectors of rigid_p and rigid_n
rigid_n = lambda x: rigid(adjoint_outermorphism(rigid, x, alg))
[terms_ratio(rigid_n(v), v) for v in sym_vecs]

[array([10.59690208, 10.59690208, 10.59690208, 10.59690208]),
 array([1.21, 1.21, 1.21, 1.21]),
 array([1.21, 1.21, 1.21, 1.21]),
 array([18.9685364, 18.9685364, 18.9685364, 18.9685364])]

In [110]:
# the same spectral decomposition of the normal transformation
beta = [v ^ rigid(v) for v in sym_vecs]
rigid_eigs = [a + b for a,b in zip(sym_eigs, beta)]
rigid(x), sum([P(x, a) * l for a, l in zip(sym_vecs, rigid_eigs)])

(1.97 𝐞₁ + -1.4 𝐞₂ + 0.972 𝐞₃ + 0.662 𝐞₄,
 1.97 𝐞₁ + -1.4 𝐞₂ + 0.972 𝐞₃ + 0.662 𝐞₄ + 3.83e-17 𝐞₁₂₃ + 1.72e-17 𝐞₁₂₄ + 1.51e-19 𝐞₁₃₄ + -1.39e-17 𝐞₂₃₄)

In [111]:
#5.19 our function "rigid" is normal but not orthogonal
A, outermorphism(rigid, adjoint_outermorphism(rigid, A, alg, h=1e-2), alg, h=1e-2)

(0.375 + 0.78 𝐞₁ + 0.427 𝐞₂ + 0.567 𝐞₃ + 0.152 𝐞₄ + 0.574 𝐞₁₂ + 0.531 𝐞₁₃ + 0.617 𝐞₁₄ + 0.686 𝐞₂₃ + 0.28 𝐞₂₄ + 0.0601 𝐞₃₄ + 0.0208 𝐞₁₂₃ + 0.596 𝐞₁₂₄ + 0.529 𝐞₁₃₄ + 0.648 𝐞₂₃₄ + 0.45 𝐞₁₂₃₄,
 0.375 + 8.31 𝐞₁ + -3.16 𝐞₂ + 1.7 𝐞₃ + -2.19 𝐞₄ + 18.5 𝐞₁₂ + -19.4 𝐞₁₃ + 34.4 𝐞₁₄ + 13.9 𝐞₂₃ + -15.6 𝐞₂₄ + -4.63 𝐞₃₄ + -77.9 𝐞₁₂₃ + 90.1 𝐞₁₂₄ + 48.4 𝐞₁₃₄ + 5.43 𝐞₂₃₄ + 1.32e+02 𝐞₁₂₃₄)

In [112]:
#5.19 But if we normalize each "eigenvalues" in the spectral decomposition
ortho_eigs = [l.normalized() for l in rigid_eigs]
ortho = lambda x: sum([P(x, a) * l for a, l in zip(sym_vecs, ortho_eigs)])
A, outermorphism(ortho, adjoint_outermorphism(ortho, A, alg, h=1e-2), alg, h=1e-2)

(0.375 + 0.78 𝐞₁ + 0.427 𝐞₂ + 0.567 𝐞₃ + 0.152 𝐞₄ + 0.574 𝐞₁₂ + 0.531 𝐞₁₃ + 0.617 𝐞₁₄ + 0.686 𝐞₂₃ + 0.28 𝐞₂₄ + 0.0601 𝐞₃₄ + 0.0208 𝐞₁₂₃ + 0.596 𝐞₁₂₄ + 0.529 𝐞₁₃₄ + 0.648 𝐞₂₃₄ + 0.45 𝐞₁₂₃₄,
 0.375 + 0.78 𝐞₁ + 0.427 𝐞₂ + 0.567 𝐞₃ + 0.152 𝐞₄ + 0.574 𝐞₁₂ + 0.531 𝐞₁₃ + 0.617 𝐞₁₄ + 0.686 𝐞₂₃ + 0.28 𝐞₂₄ + 0.0601 𝐞₃₄ + 0.0208 𝐞₁₂₃ + 0.596 𝐞₁₂₄ + 0.529 𝐞₁₃₄ + 0.648 𝐞₂₃₄ + 0.45 𝐞₁₂₃₄)

In [113]:
#5.20 left/right product of exponential
ortho(x), sum([l.inv() * P(x, a) for a, l in zip(sym_vecs, ortho_eigs)])

(0.436 𝐞₁ + -0.622 𝐞₂ + 0.894 𝐞₃ + 0.84 𝐞₄ + 3.96e-17 𝐞₁₂₃ + -4.26e-17 𝐞₁₂₄ + -6.85e-18 𝐞₁₃₄ + 1.11e-16 𝐞₂₃₄,
 0.436 𝐞₁ + -0.622 𝐞₂ + 0.894 𝐞₃ + 0.84 𝐞₄ + 2.63e-17 𝐞₁₂₃ + -6.66e-18 𝐞₁₂₄ + -6.1e-18 𝐞₁₃₄ + -1.68e-16 𝐞₂₃₄)

In [114]:
#5.21
(
    [sym_vecs[k-1].normalized() * sym_vecs[k].normalized() for k in range(1, 4)],
    [e.grade(2).normalized() for e in ortho_eigs]
)


([-1.53e-16 + 0.0499 𝐞₁₂ + 0.279 𝐞₁₃ + -0.293 𝐞₁₄ + 0.633 𝐞₂₃ + -0.637 𝐞₂₄ + 0.165 𝐞₃₄,
  -0.115 𝐞₁₂ + 0.291 𝐞₁₃ + 0.304 𝐞₁₄ + 0.636 𝐞₂₃ + 0.632 𝐞₂₄ + 0.0813 𝐞₃₄,
  -2.78e-17 + 0.165 𝐞₁₂ + 0.637 𝐞₁₃ + 0.633 𝐞₁₄ + -0.293 𝐞₂₃ + -0.279 𝐞₂₄ + 0.0499 𝐞₃₄],
 [-0.0857 𝐞₁₂ + 0.162 𝐞₁₃ + -0.179 𝐞₁₄ + -0.684 𝐞₂₃ + 0.655 𝐞₂₄ + 0.194 𝐞₃₄,
  -0.115 𝐞₁₂ + 0.291 𝐞₁₃ + 0.304 𝐞₁₄ + 0.636 𝐞₂₃ + 0.632 𝐞₂₄ + 0.0813 𝐞₃₄,
  -0.115 𝐞₁₂ + 0.291 𝐞₁₃ + 0.304 𝐞₁₄ + 0.636 𝐞₂₃ + 0.632 𝐞₂₄ + 0.0813 𝐞₃₄,
  -0.937 𝐞₁₂ + 0.281 𝐞₁₃ + -0.047 𝐞₁₄ + -0.176 𝐞₂₃ + -0.094 𝐞₂₄ + 0.037 𝐞₃₄])

In [115]:
#5.23
def simple_rotor_sqrt(R):
    # for rotations only, don't use on screw motions
    R_norm = norm(R+1)
    assert R_norm >= 1e-4, "no explicit square root for -1"
    return (R + 1)/R_norm
    
neg_eig_pairs = []
eig_rotors = []
for i, e in enumerate(ortho_eigs):
    if normsq(e + 1) <= 1e-4:
        neg_eig_pairs.append((-1, sym_vecs[i]))
    else:
        eig_rotors.append(simple_rotor_sqrt(e))

sym_vecs, [[e.reverse().sw(a) for e in eig_rotors] for a in sym_vecs]

([0.0792 𝐞₁ + 0.0531 𝐞₂ + -0.709 𝐞₃ + 0.699 𝐞₄,
  0.4 𝐞₁ + 0.898 𝐞₂ + -0.0599 𝐞₃ + -0.174 𝐞₄,
  0.174 𝐞₁ + 0.102 𝐞₂ + 0.701 𝐞₃ + 0.684 𝐞₄,
  -0.896 𝐞₁ + 0.425 𝐞₂ + 0.0468 𝐞₃ + 0.117 𝐞₄],
 [[0.0792 𝐞₁ + 0.0531 𝐞₂ + -0.709 𝐞₃ + 0.699 𝐞₄,
   0.0792 𝐞₁ + 0.0531 𝐞₂ + -0.709 𝐞₃ + 0.699 𝐞₄,
   0.0792 𝐞₁ + 0.0531 𝐞₂ + -0.709 𝐞₃ + 0.699 𝐞₄],
  [0.333 𝐞₁ + 0.489 𝐞₂ + 0.602 𝐞₃ + 0.536 𝐞₄,
   0.333 𝐞₁ + 0.489 𝐞₂ + 0.602 𝐞₃ + 0.536 𝐞₄,
   0.4 𝐞₁ + 0.898 𝐞₂ + -0.0599 𝐞₃ + -0.174 𝐞₄],
  [-0.282 𝐞₁ + -0.76 𝐞₂ + 0.364 𝐞₃ + 0.459 𝐞₄,
   -0.282 𝐞₁ + -0.76 𝐞₂ + 0.364 𝐞₃ + 0.459 𝐞₄,
   0.174 𝐞₁ + 0.102 𝐞₂ + 0.701 𝐞₃ + 0.684 𝐞₄],
  [-0.896 𝐞₁ + 0.425 𝐞₂ + 0.0468 𝐞₃ + 0.117 𝐞₄,
   -0.896 𝐞₁ + 0.425 𝐞₂ + 0.0468 𝐞₃ + 0.117 𝐞₄,
   -0.896 𝐞₁ + 0.425 𝐞₂ + 0.0468 𝐞₃ + 0.117 𝐞₄]])

In [116]:
#5.25 I selected an example with -1 as an eigenvalue, 
# so we need an extra fix for the canonical form.
theta = reduce(alg.gp, eig_rotors[::2])
(
    ortho(x),
    theta.reverse().sw(x) - (2*sum(P(x, neg_eig_pairs[i][1]) for i in range(len(neg_eig_pairs))))
)


(0.436 𝐞₁ + -0.622 𝐞₂ + 0.894 𝐞₃ + 0.84 𝐞₄ + 3.96e-17 𝐞₁₂₃ + -4.26e-17 𝐞₁₂₄ + -6.85e-18 𝐞₁₃₄ + 1.11e-16 𝐞₂₃₄,
 0.436 𝐞₁ + -0.622 𝐞₂ + 0.894 𝐞₃ + 0.84 𝐞₄)

In [117]:
#5.28
blade_exp(F1) * blade_exp(F2), blade_exp(F2) * blade_exp(F1), 

(0.651 + 0.291 𝐞₁₂ + 0.427 𝐞₁₃ + 0.169 𝐞₁₄ + -0.0716 𝐞₂₃ + -0.0656 𝐞₂₄ + -0.485 𝐞₃₄ + -0.192 𝐞₁₂₃₄,
 0.651 + 0.291 𝐞₁₂ + 0.427 𝐞₁₃ + 0.169 𝐞₁₄ + -0.0716 𝐞₂₃ + -0.0656 𝐞₂₄ + -0.485 𝐞₃₄ + -0.192 𝐞₁₂₃₄)

In [118]:
#5.29
blade_exp_ = blade_exp(F1)
u1 = P(e1, F1).normalized()
u2 = u1 | blade_exp_
blade_exp_, u1 * u2

(0.966 + 0.16 𝐞₁₂ + -0.0661 𝐞₁₃ + 0.0117 𝐞₁₄ + -0.0586 𝐞₂₃ + 0.172 𝐞₂₄ + -0.0665 𝐞₃₄,
 0.966 + 0.16 𝐞₁₂ + -0.0661 𝐞₁₃ + 0.0117 𝐞₁₄ + -0.0586 𝐞₂₃ + 0.172 𝐞₂₄ + -0.0665 𝐞₃₄)

In [119]:
# reflection as sandwich product
x - 2 * P(x, neg_eig_pairs[0][1]), -neg_eig_pairs[0][1].sw(x)

(0.975 𝐞₁ + 0.632 𝐞₂ + 0.702 𝐞₃ + 0.49 𝐞₄,
 0.975 𝐞₁ + 0.632 𝐞₂ + 0.702 𝐞₃ + 0.49 𝐞₄)

In [120]:
#5.30
theta2 = theta * neg_eig_pairs[0][1]
ortho(x), -neg_eig_pairs[0][1].sw(theta.reverse().sw(x)), -theta2.reverse().sw(x)

(0.436 𝐞₁ + -0.622 𝐞₂ + 0.894 𝐞₃ + 0.84 𝐞₄ + 3.96e-17 𝐞₁₂₃ + -4.26e-17 𝐞₁₂₄ + -6.85e-18 𝐞₁₃₄ + 1.11e-16 𝐞₂₃₄,
 0.436 𝐞₁ + -0.622 𝐞₂ + 0.894 𝐞₃ + 0.84 𝐞₄,
 0.436 𝐞₁ + -0.622 𝐞₂ + 0.894 𝐞₃ + 0.84 𝐞₄)

In [121]:
#5.37a
outermorphism(ortho, A3, alg), (-1)**(3*1)*theta2.reverse().sw(A3)

(-0.557 𝐞₁₂₃ + -0.181 𝐞₁₂₄ + -0.707 𝐞₁₃₄ + -0.396 𝐞₂₃₄,
 -0.557 𝐞₁₂₃ + -0.181 𝐞₁₂₄ + -0.707 𝐞₁₃₄ + -0.396 𝐞₂₃₄)

In [122]:
#5.37b
adjoint_outermorphism(ortho, F2, alg), (-1)**(2*1)*theta2.sw(F2)

(0.535 𝐞₁₂ + -0.191 𝐞₁₃ + 0.266 𝐞₁₄ + -0.362 𝐞₂₃ + 0.407 𝐞₂₄ + 0.034 𝐞₃₄,
 0.535 𝐞₁₂ + -0.191 𝐞₁₃ + 0.266 𝐞₁₄ + -0.362 𝐞₂₃ + 0.407 𝐞₂₄ + 0.034 𝐞₃₄)

In [123]:
#5.38
outermorphism(ortho, A3 * F2, alg), outermorphism(ortho, A3, alg) * outermorphism(ortho, F2, alg)

(0.314 𝐞₁ + 0.00641 𝐞₂ + -0.147 𝐞₃ + 0.167 𝐞₄ + 0.521 𝐞₁₂₃ + -0.244 𝐞₁₂₄ + -0.434 𝐞₁₃₄ + 0.154 𝐞₂₃₄,
 0.314 𝐞₁ + 0.00641 𝐞₂ + -0.147 𝐞₃ + 0.167 𝐞₄ + 0.521 𝐞₁₂₃ + -0.244 𝐞₁₂₄ + -0.434 𝐞₁₃₄ + 0.154 𝐞₂₃₄)

In [124]:
#5.39
from scipy.stats import ortho_group
matrix = ortho_group.rvs(dim=5)
alg = Algebra(5)
ortho = matrix2trans(matrix, alg)
x = random_r_vectors(1, alg)[0]
ortho_curl = curl(ortho, x, alg, grade=1)
ortho_curl ** 2

-2.97 + 3.47e-18 𝐞₁₂ + 1.39e-17 𝐞₁₅ + -1.3e-18 𝐞₂₃ + -3.47e-18 𝐞₂₄ + -0.13 𝐞₁₂₃₄ + -0.615 𝐞₁₂₃₅ + -0.423 𝐞₁₂₄₅ + 0.693 𝐞₁₃₄₅ + -0.00971 𝐞₂₃₄₅

In [125]:
# check orthogonality
x, ortho(adjoint_outermorphism(ortho, x, alg, h=1e-2))

(0.337 𝐞₁ + 0.633 𝐞₂ + 0.932 𝐞₃ + 0.974 𝐞₄ + 0.43 𝐞₅,
 0.337 𝐞₁ + 0.633 𝐞₂ + 0.932 𝐞₃ + 0.974 𝐞₄ + 0.43 𝐞₅)

In [126]:
# only a single vector left in the dual of skew blades, 
# so it's the remaining eigenvector for the symmetric part
# with eigenvalue 1 or -1: identity or a reflection

skew_blades = bivector_split(ortho_curl/2, alg)
sym_vec = reduce(alg.ip, skew_blades, alg.pseudoscalar((1,)))
sym_vec = sym_vec / norm(sym_vec)
p = 0
if max_diff(ortho(sym_vec), -sym_vec) < 1e-6:
    p += 1

In [127]:
rotors = []
for blade in skew_blades:
    t = P(sum(alg.frame), blade)
    # blade_norm = norm(blade)
    # we can't simply rely on ortho_m:
    # angle = np.arcsin(blade_norm) has no unique solution in [0, pi].
    # (pi - angle) generates the same sine, but differnt cosine (alpha).
    
    alpha = np.median(terms_ratio(ortho(t) - (t | ortho_curl)/2, t))
    # half_blade_ortho = (angle/blade_norm/2) * blade.reverse()
    # rotors.append(blade_exp(half_blade_ortho))
    rotors.append(simple_rotor_sqrt(alpha + blade))
rotor = reduce(alg.gp, rotors)
if p == 1:
    rotor2 = sym_vec * rotor
else:
    rotor2 = rotor
(-1) ** p * rotor2.reverse().sw(x), ortho(x)

(0.114 𝐞₁ + -0.287 𝐞₂ + 0.0868 𝐞₃ + -0.991 𝐞₄ + -1.2 𝐞₅ + 4.86e-16 𝐞₁₂₃₄₅,
 0.114 𝐞₁ + -0.287 𝐞₂ + 0.0868 𝐞₃ + -0.991 𝐞₄ + -1.2 𝐞₅)

In [128]:
# On the other hand, we can solve the rotor part by Lasenby's method
frame = [item for blade in skew_blades for item in blade_split(blade, alg)]
r_frame = reciprocal(frame)
lasenby_rotor = sum(char_multi_linear(ortho, s, alg, frame, r_frame) for s in range(5))
terms_ratio(rotor, lasenby_rotor)

array([0.94191318, 0.94191318, 0.94191318, 0.94191318, 0.94191318,
       0.94191317, 0.94191317, 0.94191317, 0.94191317, 0.94191317,
       0.94191317, 0.94191318, 0.94191318, 0.94191318, 0.94191318,
       0.94191317])

In [129]:
# For orthogonal transform, we simply normalize it
l_rotor = lasenby_rotor/norm(lasenby_rotor)
if p == 1:
    l_rotor2 = l_rotor * sym_vec
else:
    l_rotor2 = l_rotor
(-1) ** p * l_rotor2.reverse().sw(x), ortho(x)

(0.114 𝐞₁ + -0.287 𝐞₂ + 0.0868 𝐞₃ + -0.991 𝐞₄ + -1.2 𝐞₅ + 8.33e-17 𝐞₁₂₃₄₅,
 0.114 𝐞₁ + -0.287 𝐞₂ + 0.0868 𝐞₃ + -0.991 𝐞₄ + -1.2 𝐞₅)

In [130]:
# Because sym_vec is orthogonal to rotor, we can extract sym_vec from l_rotor:
terms_ratio(l_rotor2.grade(1), sym_vec), terms_ratio(l_rotor | alg.pseudoscalar((1,)), sym_vec)

(array([-0., -0., -0.,  0., -0.]),
 array([0.48335845, 0.48335845, 0.48335845, 0.48335845, 0.48335845]))

In [131]:
# From Graded Symmetry Groups: Plane and Simple
def tangent_split(R, alg: Algebra):  
    m = alg.d // 2
    if m <= 1:
        return F
    Ck = [R.grade(2*r) for r in range(m+1)]
    Ck2 = [(-1)**(m - i) * alg.sp(Ck[i], Ck[i])[0] for i in range(m+1)]
    Lk = np.roots(Ck2)
    tangents = []
    for l in Lk:
        even = sum(l ** r * c for r, c in enumerate(Ck[-1::-2]))
        odd = sum(l ** r * c for r, c in enumerate(Ck[-2::-2]))
        tangents.append(even / odd) # FIXME: the inverse
    return tangents

# tangents = tangent_split(l_rotor, alg)
# rotors2 = []
# for t in tangents:
#     r = 1+t
#     rotors2.append(r/norm(r))
# rotors2, rotors 